In [ ]:
from getpass import getpass

In [ ]:
!pip install langchain openai faiss-cpu tiktoken



In [ ]:
import os
os.environ['OPENAI_API_KEY']=getpass()

··········


In [ ]:
from operator import itemgetter

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.vectorstores import FAISS

In [ ]:
vectorstore = FAISS.from_texts(["parivesh worked at Interview Kickstart"], embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
chain.invoke("where did parivesh work?")



'Parivesh worked at Interview Kickstart.'

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = {
    "context": itemgetter("question") | retriever,
    "question": itemgetter("question"),
    "language": itemgetter("language")
} | prompt | model | StrOutputParser()

In [ ]:
chain.invoke({"question": "where did parivesh work", "language": "italian"})



'Parivesh ha lavorato a Interview Kickstart.'